Import all the necessary libraries

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import warnings 
warnings.filterwarnings('ignore')
from torch.utils.data import DataLoader
from torchvision import datasets ,transforms
from torchvision.transforms import ToTensor  ####convert tensors to vectors
from tensorflow.keras.layers import Dropout

Downloading the data and converting the data into tensor format

In [ ]:
train = datasets.MNIST(root = 'data' , train = True,download = True,transform = ToTensor())
test = datasets.MNIST(root = 'data' , train = False,download = True,transform = ToTensor())

Visualizing some data points from the data set

In [ ]:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train), size=(1,)).item()
    img, label = train[sample_idx]
    figure.add_subplot(rows, cols, i)
    #plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

Converting the data into small batches using data loader show that you can pass your data in small portions instead of sending it wants once 

In [ ]:
batch_size = 64
train_data = DataLoader(train,batch_size = batch_size)
test_data = DataLoader(test,batch_size = batch_size)

Visualizing first data point import tensor format image format

In [ ]:
for X,y in train_data:
    print (X)
    print (y)
    plt.imshow(X[5].squeeze(),cmap = 'Greens') 
    #plt.imshow(X[5].squeeze(),cmap = 'gray')
    break

check size and shape  shape of the data 

In [ ]:
len(train_data),len(test_data)

In [ ]:
for x,y in train_data:
    print(x.shape)
    print(y.shape)
    print(x[0].shape)
    #print(y[0].shape)
    break

if GPS is available then use GPU else  use CPU 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# Neural architecture using traditional method

In [ ]:
class neural_network(nn.Module):
    def __init__(self):
        super(neural_network,self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(28*28,512)
        self.layer2 = nn.Linear(512,256)
        self.layer3 = nn.Linear(256,512)
        self.layer4 = nn.Linear(512,128)
        self.out = nn.Linear(128,10)
        #self.dropout = nn.Dropout(0.25)
    def forward(self,x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = F.relu(x)
        #x = self.dropout(x)
        x = self.layer2(x)
        x = F.relu(x)
        #x = self.dropout(x)
        x = self.layer3(x)
        x = F.relu(x)
        x = self.layer4(x)
        x = F.relu(x)
        #x = self.dropout(x)
        x = self.out(x)
        x = F.softmax(x)
        return x
        

In [ ]:
model = neural_network()
model = model.to(device)

In [ ]:
model

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr= 0.01)

In [ ]:
def train(data,model,loss_,optim):
    model.train()
    for batch ,(X,y) in enumerate(data):
        X = X.to(device)
        y = y.to(device)
        #print(X)
    
        pred = model(X)
        loss = loss_(pred,y)
    
    
        loss.backward()
        optim.step()
        optim.zero_grad()
        if batch % 100 == 0:
            print(batch)
            print(f'loss:{loss}')

In [ ]:
# train(train_data,model,loss_fn,optimizer)

In [ ]:
def test(model,data,loss_):
    model.eval()
    test_error,correct = 0,0
    size  = len(data.dataset)
    
    with torch.no_grad():
        
        for batch,(X,y) in enumerate(data):
            X = X.to(device)
            y = y.to(device)
    
            pred = model(X)
            test_error += loss_(pred,y).item()
            #print(test_error)
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
        print(f'test_correct {correct} out of {size} and  Accuracy is{correct/size*100}' )
        print('test error :::',test_error)

In [ ]:
# test(model,test_data,loss_fn)

In [ ]:
epoch = 10
i = 1
for t in range(epoch):
    print('-----------------epoch number -----',i)
    train(train_data,model,loss_fn,optimizer)
    test(model,test_data,loss_fn)
    i+=1

# Neural architecture using Dropout,regulizer,early_stop methods

In [ ]:
class neural_network(nn.Module):
    def __init__(self):
        super(neural_network,self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(28*28,512)
        self.layer2 = nn.Linear(512,256)
        self.layer3 = nn.Linear(256,512)
        self.layer4 = nn.Linear(512,128)
        self.out = nn.Linear(128,10)
        self.dropout = nn.Dropout(0.25)
    def forward(self,x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = F.relu(x)
        x = self.layer4(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.out(x)
        x = F.relu(x)
        return x
        

In [ ]:
model = neural_network()
model = model.to(device)
model

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr= 0.01,weight_decay = 0.001)

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [ ]:
def train(data,model,loss_,optim):
    model.train()
    for batch ,(X,y) in enumerate(data):
        X = X.to(device)
        y = y.to(device)
        #print(X)
    
        pred = model(X)
        loss = loss_(pred,y)
    
    
        loss.backward()
        optim.step()
        optim.zero_grad()
        writer.flush()
        if batch % 150 == 0:
            print(batch)
            print(f'loss:{loss}')

In [ ]:
def test(model,data,loss_):
    model.eval()
    test_error,correct = 0,0
    size  = len(data.dataset)
    
    with torch.no_grad():
        
        for batch,(X,y) in enumerate(data):
            X = X.to(device)
            y = y.to(device)
    
            pred = model(X)
            test_error += loss_(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
        test_new = test_error
        print(f'test_correct {correct} out of {size} and  Accuracy is{correct/size*100}' )
        print('test error :::',test_error)
    return test_new

In [ ]:
epoch = 100

i = 1
test_old = float('inf')
for t in range(epoch):
    print('-----------------epoch number -----',i)
    train(train_data,model,loss_fn,optimizer)
    test_new = test(model,test_data,loss_fn)
    print (f'test_new {test_new}')
    print (f'test_old {test_old}')
    print (f'test_difference {test_old - test_new}')
    i += 1
    if abs(test_old - test_new) <= 0.1:
        print('condition reached and the value is :::;;',test_new - test_old)
        break
    else :
        test_old = test_new

In [ ]:
print('done')
     

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./runs2')
%load_ext tensorboard

In [ ]:
tensorboard --logdir


In [ ]:
pip install tensorflow tensorboard


In [ ]:
import tensorboard
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
!kill 584

In [ ]:
#Now go back to your tensorboard screen, and you will see the result. Or

%tensorboard --logdir logs